# Complex Control Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/control_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

[EPyT-Flow](https://github.com/WaterFutures/EPyT-Flow) is available on [PyPI](https://pypi.org/project/epyt-flow/) and can be installed via `pip install epyt-flow`:

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
from epyt.epanet import ToolkitConstants
from epyt_flow.data.networks import load_net1
from epyt_flow.simulation import ScenarioSimulator
from epyt_flow.simulation.scada import ComplexControlModule, RuleAction, RuleCondition, \
    EN_R_LEVEL, EN_R_LEQ, EN_R_GEQ, EN_R_ACTION_STATUS_CLOSED, EN_R_ACTION_STATUS_OPEN
from epyt_flow.utils import to_seconds

Create two complex control modules (i.e. entries in the "RULES" section of an .inp file) for controlling pump "9" based on the water leven in tank "2":

- IF TANK 2 LEVEL <= 110 THEN PUMP 9 SETTING IS OPEN
- IF TANK 2 LEVEL >= 140 THEN PUMP 9 SETTING IS CLOSED

The first control rule:
IF TANK 2 LEVEL <= 110 THEN PUMP 9 SETTING IS OPEN

1. Create the condition (instance of [RuleCondition](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.complex_control.RuleCondition)) TANK 2 LEVEL <= 110
2. Create the action (instance of [RuleAction](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.complex_control.RuleAction)): PUMP 9 SETTING IS OPEN
3. Combine condition and action into the final control rule (instance of [ComplexControlModule](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.complex_control.ComplexControlModule))

In [ ]:
# IF TANK 2 LEVEL <= 110 THEN PUMP 9 SETTING IS OPEN
# Create the condition: TANK 2 LEVEL <= 110
condition_1 = RuleCondition(object_type_id=ToolkitConstants.EN_R_NODE,
                            object_id="2",
                            attribute_id=EN_R_LEVEL,
                            relation_type_id=EN_R_LEQ,
                            value=110)

# Create the action: PUMP 9 SETTING IS OPEN
action_1 = RuleAction(link_type_id=ToolkitConstants.EN_PUMP,
                      link_id="9",
                      action_type_id=EN_R_ACTION_STATUS_OPEN,
                      action_value=None)

# Combine condition and action into the final control rule
my_control_1 = ComplexControlModule(rule_id="PUMP-9_1",
                                    condition_1=condition_1,
                                    additional_conditions=[],
                                    actions=[action_1],
                                    else_actions=[],
                                    priority=1)

The second control rule:
IF TANK 2 LEVEL >= 140 THEN PUMP 9 SETTING IS CLOSED

1. Create the condition (instance of [RuleCondition](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.complex_control.RuleCondition)) TANK 2 LEVEL >= 140
2. Create the action (instance of [RuleAction](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.complex_control.RuleAction)): PUMP 9 SETTING IS CLOSED
3. Combine condition and action into the final control rule (instance of [ComplexControlModule](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.complex_control.ComplexControlModule))

In [ ]:
# IF TANK 2 LEVEL >= 140 THEN PUMP 9 SETTING IS CLOSED
# Create condition: TANK 2 LEVEL >= 140
condition_1 = RuleCondition(object_type_id=ToolkitConstants.EN_R_NODE,
                            object_id="2",
                            attribute_id=EN_R_LEVEL,
                            relation_type_id=EN_R_GEQ,
                            value=140)

# Create action: TANK 2 LEVEL >= 140
action_1 = RuleAction(link_type_id=ToolkitConstants.EN_PUMP,
                      link_id="9",
                      action_type_id=EN_R_ACTION_STATUS_CLOSED,
                      action_value=None)

# Combine condition and action into the final control rule
my_control_2 = ComplexControlModule(rule_id="PUMP-9_2",
                                    condition_1=condition_1,
                                    additional_conditions=[],
                                    actions=[action_1],
                                    else_actions=[],
                                    priority=1)

Create new simulation based on Net1:

In [ ]:
sim = ScenarioSimulator(scenario_config=load_net1(verbose=False))

Set simulation duration to two days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Monitor states of tank "2" and pump "9":

In [ ]:
sim.set_tank_sensors(sensor_locations=["2"])
sim.set_pump_state_sensors(sensor_locations=["9"])

Note that Net1.inp contains some simple controls. Remove all of them by calling [remove_all_simple_controls()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.remove_all_simple_controls):

In [ ]:
sim.remove_all_simple_controls()

Add our complex control modules by calling [add_complex_control()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_complex_control):

In [ ]:
sim.add_complex_control(my_control_1)
sim.add_complex_control(my_control_2)

Run the simulation and show sensor readings over time:

In [ ]:
scada_data = sim.run_simulation()

In [ ]:
scada_data.plot_pumps_state()

In [ ]:
scada_data.plot_tanks_water_volume()

Do not forget to close the simulation!

In [ ]:
sim.close()